In [ ]:
# ANN, MLP과 관련된 내용들을 다뤘음
# ANN이 training 되는 과정에서 Gradient Descent, Backpropagation이 작동됨 
# 추가로 ANN을 효과적으로 활용하기 위해서 Regularization, Optimization 방법이 있음

import numpy as np
import torch
from torch import nn             # nn-> 신경망을 활용하기 위해 필요한 module
import matplotlib.pyplot as plt  # logistic regression을 시각화하기 위함 
%matplotlib inline

## Logistic Regression (PyTorch)

In [ ]:
# 데이터를 생성해서 점들로 표현한 후, 점들을 classification 하는 문제
# 1000개씩 2줄 형태로 data setting 
# X0와 X1을 구분해서 생성 -> 분류를 위함 
# X0 -> normal distribution, random하게 정의, 평균이 2가 됨 
# X1 -> 평균이 -2가 되는 data set 
# y0 -> 0, y1 -> 1 

n_data = torch.ones(1000, 2)

X0 = torch.normal(2 * n_data, 1)
y0 = torch.zeros(1000)
X1 = torch.normal(-2 * n_data, 1)
y1 = torch.ones(1000)


# vstack을 활용하여 X,y끼리 묶어줌 
# train_y가 true/false 인 case로 정의 
train_X = np.vstack([X0, X1])
train_y = np.vstack([y0, y1]).reshape(-1, 1)

C1 = np.where(train_y == True)[0]
C0 = np.where(train_y == False)[0]


# Define train_X, train_y for torch
# torch를 활용한 training을 위해 이에 맞는 형태로 data define 
# from_numpy() -> numpy로부터 data를 받음, X data가 normal distribution을 따르는 random한 숫자들을 포함하므로 float형태로 나옴
# 2000개의 data 중 1000개는 true, 1000개는 false와 관련된 data
train_X, train_y = torch .from_numpy(train_X).float(), torch.from_numpy(train_y).float() 

print(train_X.shape, train_y.shape)

In [ ]:
# data plot 
# C0 -> x1, x2 도 평균이 2정도, 표준편차가 1이 되는 data들
# C1 -> x1, x2 모두 -2를 평균으로 갖는, normal distribution을 따르는 data들 
# C1, C2를 하나의 hyper plane을 통해서 두 개의 data를 classification하는 것 

plt.figure(figsize = (10,8))
plt.plot(train_X[C1,0], train_X[C1,1], 'ro', alpha = 0.3, label='C1')
plt.plot(train_X[C0,0], train_X[C0,1], 'bo', alpha = 0.3, label='C0')
plt.xlabel(r'$x_1$', fontsize = 15)
plt.ylabel(r'$x_2$', fontsize = 15)
plt.legend(loc = 1, fontsize = 12)
plt.axis('equal')
plt.ylim([-5,5])
plt.show()

In [ ]:
# modeling에 적용하기 위해서는 dataloader, tensordataset을 활용해주어야 함 
# TensorDataset : 학습하고자 하는 data들, tensor들을 찾는 과정 -> 일반적으로 x, y data가 많이 포함됨 
# 우리는 x,y를 data array 형태로 받음 
# dataset을 data array형태로 TensorDataset을 받아옴 
# dataloader는 어떤 dataset을 쓰고, batch size, shuffle 어떻게 되는지 포함하는 function 

from torch.utils.data import DataLoader, TensorDataset

def load_array(data_arrays, batch_size, is_train = True):
    # Define dataset and dataloader
    dataset = TensorDataset(*data_arrays)
    dataloader = DataLoader(dataset = dataset, 
                            batch_size = batch_size,
                            shuffle = is_train)
    return dataloader

In [ ]:
# train_X, train_y를 batch size는 나누지 않고 전체 데이터를 통으로 data iteration을 적용
# 학습을 하기 위한 형태로 데이터를 정리함 

data_iter = load_array((train_X, train_y), batch_size=len(train_y))

In [ ]:
# torch.nn Module class를 상속받아 Logistic Regression Mdel class를 사용할 수 있게 됨
# 새로운 DL model을 설계하는데 효과적으로 활용될 수 있음 
# init : Logistic rigression model class의 instance를 생성했을 때 갖게 되는 성질 정의


class LogisticRegressionModel(torch.nn.Module):
    # Define init, forward method
    # NN model 내 method를 상속받게 함
    # layer는 처음에 Linear, input later ~ output layer 뉴런의 수*노드의 수) 입력 
    # X1 X2를 받아서 y를 맞춰야 하므로 2, 1
    # linear로 결합된 값을 sigmoid function을 통해서 최종적으로 결론이 난다 
    def __init__(self):
        super(LogisticRegressionModel, self).__init__()  
        self.layer = torch.nn.Linear(2, 1)              
        self.sigmoid = torch.nn.Sigmoid()

    # 하나 하나의 layer들의 관계를 정의해 줌 
    # 도출되는 output은 self.layer의 input을 넣었을 때가 됨 
    # 첫 번째 layer를 통과한 output은 sigmoid function을 통과시킴 
    def forward(self, inputs):
        outputs = self.layer(inputs)
        return self.sigmoid(outputs)

model_logR = LogisticRegressionModel()

# cuda, GPU를 추가적으로 사용할지 묻는 코드 
# GPU를 사용한다면 train_X, train_y, model까지 GPU를 사용할 수 있도록 정의해주어야 함
#if torch.cuda.is_available():
#    train_X, train_y = train_X.cuda(), train_y.cuda()
#model_logR.cuda()

In [ ]:
# model log 아래 layer들의 weight와 bias를 initialize한다 

print(model_logR.layer.weight.data)
print(model_logR.layer.bias.data)

In [ ]:
# Define SGD optimizer 
# torch의 optimizer를 나타내는 module 밑의 SGD function을 이용

optimizer_logR = torch.optim.SGD(model_logR.parameters(), lr = 0.05)

In [ ]:
# parameters : layer와 어떤 activation function을 사용하는지 나옴 

model_logR.parameters

In [ ]:
# Training 단계

num_epochs = 200
loss_graph_logR = []

# Train X data를 model에 넣고 binary cross entropy를 통해 loss를 계산한 후 
# 최적화에 대해서 gradient를 초기화한 후 loss에 대해 backward을 진행, optimizer에 대해서 관련된 weight들을 update한다
# X를 넣어서 y를 도출하고 model을 통해 도출된 y와 train_y를 비교해서 loss를 구한 후 
# gradient를 초기화, backpropagation, update하는 순서로 진행함 
for epoch in range(num_epochs):
    for X, y in data_iter:
        # training with predict, loss, zero_grad, backward, step
        predict_logR = model_logR(train_X)
        loss_logR = torch.nn.functional.binary_cross_entropy(predict_logR, train_y)
        optimizer_logR.zero_grad()
        loss_logR.backward()
        optimizer_logR.step()
    loss_graph_logR.append(torch.nn.functional.binary_cross_entropy(model_logR(train_X), train_y))

plt.plot(loss_graph_logR)
plt.xlabel("epoch")
plt.ylabel("loss")
plt.show()

In [ ]:
# logistic regression model에 대한 weight를 구함 -> x1, x2에 대한 weight 
# bais에 대한 weight 
w1 = model_logR.layer.weight[0][0].item()
w2 = model_logR.layer.weight[0][1].item()
b = model_logR.layer.bias.item()

print(w1, w2, b)

xp = np.arange(-4, 4, 0.01).reshape(-1, 1)
yp = - w1 / w2 * xp - b / w2

# 앞서 정의한 train_x,y를 plot하기 위해 pug로 만든 것을 cpu로 되돌림
train_X, train_y = train_X.cpu(), train_y.cpu()

plt.figure(figsize = (10,8))
plt.plot(train_X[C1,0], train_X[C1,1], 'ro', alpha = 0.3, label='C1')
plt.plot(train_X[C0,0], train_X[C0,1], 'bo', alpha = 0.3, label='C0')
plt.plot(xp, yp, 'g', linewidth = 3, label = 'Logistic Regression')
plt.xlabel(r'$x_1$', fontsize = 15)
plt.ylabel(r'$x_2$', fontsize = 15)
plt.legend(loc = 1, fontsize = 12)
plt.axis('equal')
plt.ylim([-4,4])
plt.show()

# 두 개의 다른 dataset을 분류하기 위한 logistic regression을 돌렸고 
# 학습된 결과로 관련된 weight, W1, W2, bias와 관련된 weight가 도출됨 

## MNIST_MLP

In [ ]:
# MLIST를 통해서 MLP 구현 
# image data -> torch vision 
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch import nn
from torch.nn import functional as F
from torchvision import transforms, datasets
from torch.utils.data import DataLoader

In [ ]:
BATCH_SIZE = 32
EPOCHS = 10

In [ ]:
# torch vision 밑에 dataset이 존재하는데 그 중 MLIST dataset을 불러옴   

train_dataset = datasets.MNIST(root = "../data/MNIST",
                               train = True,
                               download = True,
                               transform = transforms.ToTensor())

test_dataset = datasets.MNIST(root = "../data/MNIST",
                              train = False,
                              transform = transforms.ToTensor())

# Define train_loader, Test_loader
# data를 loader 형태로 변환 
# test 는 shuffle하지 않고 그대로 사용 
train_loader = DataLoader(dataset=train_dataset,
                          batch_size = BATCH_SIZE,
                          shuffle = True)

test_loader = DataLoader(dataset=test_dataset,
                         batch_size = BATCH_SIZE,
                         shuffle = False)

In [ ]:
# size 파악 
# batch size를 32로 했기 때문에 하나의 mini batch가 32개의 image data로 구성되어 있음을 알 수 있음 
# 각각의 image는 28*28 형태 
# y_train는 image에 해당하는 label만 포함되어 있기 때문에 단순히 batch size만큼만 할당됨 
# FloatTensor / LongTensor 

for (X_train, y_train) in train_loader:
    print('X_train:', X_train.size(), 'type:', X_train.type())
    print('y_train:', y_train.size(), 'type:', y_train.type())
    break

In [ ]:
# input이 어떤 형태로 들어있는가? 

pltsize = 1
plt.figure(figsize=(10 * pltsize, pltsize))
for i in range(10):
    plt.subplot(1, 10, i + 1)
    plt.axis('off')
    plt.imshow(X_train[i, :, :, :].numpy().reshape(28, 28), cmap = "gray_r")
    plt.title('Class: ' + str(y_train[i].item()))

In [ ]:
# MLP 정의하는 단계 -> NeuralNet이라는 class로 정의  

class NeuralNet(torch.nn.Module):
    # Define init and forward method
    # nn module에 있는 method를 상속받아 사용하기 위한 super 
    # fully connected layer 총 3개를 정의 
    # 1) linear, input이 28*28 -> 512개 형태의 output 만듬
    # 2) 512개의 input을 받아서 256개의 output 도출
    # 3) 256개의 input을 받아서 10개의 output(숫자, 0~9)도출  
    def __init__(self):
        super(NeuralNet, self).__init__()
        self.fclayer1 = torch.nn.Linear(28 * 28, 512)
        self.fclayer2 = torch.nn.Linear(512, 256)
        self.fclayer3 = torch.nn.Linear(256, 10)
        # drop out probability 0.5, 각각의 sigmoid function 후에 적용됨 
        # 참고로 drop out을 적용할 때는 sigmoid보다 relu가 좋기 때문에 relu로 적용 
        self.dropout_prob = 0.5
        # batch normalization -> activation function 앞에 적용
        self.batch_norm1 = nn.BatchNorm1d(512)
        self.batch_norm2 = nn.BatchNorm1d(256)

    # 28*28 input을, 2차원 data를 1차원 data로 변환하는 과정 (view)
    # fcl1과 연결 -> sigmoid function연결 (activation function)
    # 다시 fc2와 연결 -> 다시 sigmoide로 연결 
    # 다시 fc3와 연결 -> 마지막으로 최종적인 output이 10개가 나오게 하는 log_softmax function을 활용
    # x return 
    def forward(self, x):
        x = x.view(-1, 28 * 28)
        x = self.fclayer1(x)
        # 첫 번째 batch normalization 
        x = self.batch_norm1(x)
        x = F.relu(x)
        # 첫 번째 sigmoid function 이후 dropout, input은 x
        x = F.dropout(x, training=self.training, p = self.dropout_prob)
        x = self.fclayer2(x)
        # 두 번째 batch normalization 
        x = self.batch_norm2(x)
        x = F.relu(x)
        # 두 번째 sigmoid function 이후 dropout 
        x = F.dropout(x, training=self.training, p = self.dropout_prob)
        x = self.fclayer3(x)
        x = F.log_softmax(x, dim = 1)
        return x

In [ ]:
# train, validation도 해야 하기 때문에 device형태로 gpu 사용 유무를 받음 
if torch.cuda.is_available():
    DEVICE = torch.device('cuda')
else:
    DEVICE = torch.device('cpu')

In [ ]:
# optimizer와 loss 정의 

# model instance화 
model = NeuralNet().to(DEVICE)

# Define optimizer
# optim.SGD(model.parameters(), lr = 0.01, momentum=0.5)
optimizer = torch.optim.Adam(model.parameters(), lr = 0.01)

# Define loss 
criterion = nn.CrossEntropyLoss()

print(model)

In [ ]:
# training 

def train(model, train_loader, optimizer, log_interval):
    model.train()
    for batch_idx, (image, label) in enumerate(train_loader):
        # image, label을 gpu를 쓰게 되면 쓰게끔 하는 명령어 
        image = image.to(DEVICE)
        label = label.to(DEVICE)
        # training with output, loss, zero_grad, backward, step
        output = model(image)
        loss = criterion(output, label)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
   
        if batch_idx % log_interval == 0:
            print("Train Epoch: {} [{}/{} ({:.0f}%)]\tTrain Loss: {:.6f}".format(
                epoch, batch_idx * len(image), 
                len(train_loader.dataset), 100. * batch_idx / len(train_loader), 
                loss.item())) 

In [ ]:
# evaluation 

def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0

    # 여기서 중요한 것은 torch가 weight를 update하기 위해서 gradient를 활용하지 못하게 정의해야 함 
    with torch.no_grad():
        for image, label in test_loader:
            image = image.to(DEVICE)
            label = label.to(DEVICE)
            # Evaluate with output, test_loss, prediction, correct
            # test_loss와 correct에 대해서 관련된 값들을 누적시킬 것임 
            # test_loss -> 마지막 test_loader의 lenght로 나누어 계산할 것임 -> 계속 누적하면 됨 
            # prediction -> 도출된 output에 대해서 (각각의 label에 대한 확률로 나올 것임) 그 중 가장 높은 확률을 prediction 결과값으로 도출
            # correct -> accuracy 계산을 위함, test_loader.length로 나누어줄 것이기 때문에 누적 형태로 계산 
            # 도출된 prediction이 원래 값과 동일한지 아닌지를 확인하는 단계가 됨
            output = model(image)
            test_loss += criterion(output, label).item()
            prediction = output.max(1, keepdim = True)[1]
            correct += prediction.eq(label.view_as(prediction)).sum().item()
    
    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

In [ ]:
# 직접 train, evaluation 진행 
# epoch 10번 당 한 번의 evaluation
# drop_out, batch normalization, optimizer -> Adam 3개가지만 적용 -> 90% -> 97% 성능 상승 
# 단순한 fully connected layer -> 90%, Regularization -> 적절한 일반화 과정이 필요합니다! 

for epoch in range(1, EPOCHS + 1):
    train(model, train_loader, optimizer, log_interval = 200)
    test_loss, test_accuracy = evaluate(model, test_loader)
    print("\n[EPOCH: {}], \tTest Loss: {:.4f}, \tTest Accuracy: {:.2f} % \n".format(
        epoch, test_loss, test_accuracy))